# Bayesian Model

This notebook illustrate how to use a Bayesian Normal density model with the [beer framework](https://github.com/beer-asr/beer). The Normal distribution is a fairly basic models but it is used extenslively in other model as a basic building block.

In [7]:
# Add "beer" to the PYTHONPATH
import sys
sys.path.insert(0, '../')

import beer
import numpy as np
import torch

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
output_notebook()

# Convenience functions for plotting.
import plotting

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data

Generate some normally distributed data:

In [8]:
mean = np.array([-1.5, 4]) 
cov = np.array([
    [2, 1],
    [1, .75]
])
data = np.random.multivariate_normal(mean, cov, size=100)

fig = figure(
    title='Data',
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1])
plotting.plot_normal(fig, mean, cov, alpha=.5, color='black')

show(fig)

## Model Creation

We create two types of Normal distribution: one diagonal covariance matrix and another one with full covariance matrix.

In [9]:
normal_diag = beer.NormalDiagonalCovariance.create(
    mean=torch.zeros(2),
    diag_cov=torch.ones(2)
)
normal_full = beer.NormalFullCovariance.create(
    mean=torch.zeros(2),
    cov=torch.eye(2)
)

## Variational Bayes Training 

In [10]:
epochs = 1
lrate = 1
X = torch.from_numpy(data).float()
elbo_fn = beer.EvidenceLowerBound(len(X))
params = normal_diag.parameters + normal_full.parameters
models = [normal_diag, normal_full]
optimizer = beer.BayesianModelOptimizer(params, lrate)
    
for epoch in range(epochs):
    optimizer.zero_grad()
    for model in models:
        elbo = elbo_fn(model, X)
        elbo.natural_backward()
    optimizer.step()

In [11]:
fig = figure(
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1])
plotting.plot_normal(fig, normal_diag.mean.numpy(), normal_diag.cov.numpy(), alpha=.5, color='red')
plotting.plot_normal(fig, normal_full.mean.numpy(), normal_full.cov.numpy(), alpha=.5, color='blue')

show(fig)